# 01 - YOLO Tradicional

Teste rápido do YOLO pré-treinado (zero setup).

## Setup

In [ ]:
import sys
sys.path.append('../src')

from pathlib import Path
import time
import numpy as np
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
import torch

from utils.visualization import plot_sample_images
from utils.metrics import Timer

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

## Carregar YOLO

In [ ]:
model = YOLO('yolov8n.pt')
print(f"Modelo carregado: {len(model.names)} classes do COCO")

## Testar em Imagens

In [ ]:
ROOT_DIR = Path('../')
coffee_test = list((ROOT_DIR / 'Coffee' / 'Teste').glob('*.jpg'))[:4]
mountains_test = list((ROOT_DIR / 'Mountains' / 'Teste').glob('*.jpg'))[:4]

print(f"Testando em {len(coffee_test + mountains_test)} imagens...")

results = model.predict(coffee_test + mountains_test, conf=0.25, device=DEVICE)
print("Pronto!")

## Visualizar Resultados

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, (result, ax) in enumerate(zip(results, axes)):
    img = result.plot()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    ax.imshow(img_rgb)
    ax.axis('off')
    ax.set_title(f'{len(result.boxes)} detecções')

plt.tight_layout()
plt.show()

## Medir Velocidade

In [ ]:
test_img = str(coffee_test[0])

# Warm-up
_ = model.predict(test_img, device=DEVICE, verbose=False)

# Medir
times = []
for _ in range(20):
    with Timer() as timer:
        _ = model.predict(test_img, device=DEVICE, verbose=False)
    times.append(timer.get_elapsed_time() * 1000)

print(f"Tempo médio: {np.mean(times):.2f}ms")

## Conclusão

YOLO pré-treinado:
- Muito fácil de usar
- Inferência ~200ms
- Detecta objetos gerais (cup, bottle)
- Não reconhece "coffee" ou "mountains" especificamente

Para nosso problema, precisa de fine-tuning (notebook 02).